In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5 import uic
import import_ipynb
from Kiwoom import *

form_class=uic.loadUiType("stock.ui")[0]

class MyWindow(QMainWindow, form_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        
        #변수 등록
        self.trade_stocks_done=False
        
        #evemt 등록
        self.lineEdit.textChanged.connect(self.code_changed)
        self.pushButton.clicked.connect(self.send_order)
        self.pushButton_2.clicked.connect(self.check_balance)
        
        #timer, 실시간 조회
        self.timer2=QTimer(self)
        self.timer2.start(1000*2)
        self.timer2.timeout.connect(self.timeout2)

        self.kiwoom=Kiwoom()
        self.kiwoom.comm_connect()

        self.timer = QTimer(self)
        self.timer.start(1000)
        self.timer.timeout.connect(self.timeout)

        #get account information
        accouns_num=int(self.kiwoom.get_login_info("ACCOUNT_CNT"))
        accounts=self.kiwoom.get_login_info("ACCNO")
        accounts_list=accounts.split(';')[0:accouns_num]
        self.comboBox.addItems(accounts_list)
        
        #get buy/sell list
        self.load_buy_sell_list()

    def trade_stocks(self):
        hoga_lookup={'지정가':"00",'시장가':"03"}
        
        
        f=open("buy_list.txt",'rt', encoding='UTF-8')
        buy_list=f.readlines()
        f.close()
        
        f=open("sell_list.txt",'rt', encoding='UTF-8')
        sell_list=f.readlines()
        f.close()
        
        #account
        account=self.comboBox.currentText()
        
        #매수 리스트
        for row_data in buy_list:
            split_row_data=row_data.split(';')
            hoga=split_row_data[2]
            code=split_row_data[1]
            num=split_row_data[3]
            price=split_row_data[4]
            
            if split_row_data[-1].rstrip()=='매수전':
                self.kiwoom.send_order("send_order_req","0101",account, 1, code, num, price, hoga_lookup[hoga],"")
                
        #매도 리스트
        for row_data in sell_list:
            split_row_data=row_data.split(';')
            hoga=split_row_data[2]
            code=split_row_data[1]
            num=split_row_data[3]
            price=split_row_data[4]
            
            if split_row_data[-1].rstrip()=='매도전':
                self.kiwoom.send_order("send_order_req","0101",account, 2, code, num, price, hoga_lookup[hoga],"")
                
        
        #파일 정리(매수)
        for i, row_data in enumerate(buy_list):
            buy_list[i]=buy_list[i].replace("매수전","주문완료")
        
        f=open("buy_list.txt",'wt',encoding='UTF-8')
        for row_data in buy_list:
            f.write(row_data)
        f.close()
        
        #파일 정리(매도)
        for i, row_data in enumerate(sell_list):
            sell_list[i]=sell_list[i].replace("매도전","주문완료")
        
        f=open("sell_list.txt",'wt',encoding='UTF-8')
        for row_data in sell_list:
            f.write(row_data)
        f.close()
        
    def load_buy_sell_list(self):
        
        f=open("buy_list.txt",'rt', encoding='UTF-8')
        buy_list=f.readlines()
        f.close()
        
        f=open("sell_list.txt",'rt', encoding='UTF-8')
        sell_list=f.readlines()
        f.close()
        
        row_count=len(buy_list)+len(sell_list)
        self.tableWidget_3.setRowCount(row_count)
        
        # buy_list
        for j in range(len(buy_list)):
            row_data=buy_list[j]
            split_row_data=row_data.split(';')
            split_row_data[1]=self.kiwoom.get_master_code_name(split_row_data[1].rsplit())
            
            for i in range(len(split_row_data)):
                item=QTableWidgetItem(split_row_data[i].rstrip())
                item.setTextAlignment(Qt.AlignVCenter|Qt.AlignCenter)
                self.tableWidget_3.setItem(j,i,item)
                
        # sell_list
        for j in range(len(buy_list)):
            row_data=sell_list[j]
            split_row_data=row_data.split(';')
            split_row_data[1]=self.kiwoom.get_master_code_name(split_row_data[1].rsplit())
            
            for i in range(len(split_row_data)):
                item=QTableWidgetItem(split_row_data[i].rstrip())
                item.setTextAlignment(Qt.AlignVCenter|Qt.AlignCenter)
                self.tableWidget_3.setItem(len(buy_list)+j,i,item)
                
        self.tableWidget_3.resizeRowsToContents()
        

    def timeout(self):
        market_start_time=QTime(9,0,0)
        current_time=QTime.currentTime()
        
        #자동주문
        if current_time>market_start_time and self.trade_stocks_done is False:
            #self.trade_stocks()
            self.trade_stocks_done=True
        
        
        text_time=current_time.toString("hh:mm:ss")
        time_msg="현재시간: "+ text_time

        state=self.kiwoom.get_connect_state()
        if state==1:
            state_msg="서버 연결 중..."
        else:
            state_msg="서버 미 연결 중..."

        self.statusbar.showMessage(state_msg+" | "+time_msg)

    def code_changed(self):
        code=self.lineEdit.text()
        name=self.kiwoom.get_master_code_name(code)
        self.lineEdit_2.setText(name)

    def send_order(self):
        order_type_lookup={'신규매수':1, '신규매도':2, '매수취소':3, '매도취소':4}
        hoga_lookup={'지정가':"00",'시장가':"03"}

        account=self.comboBox.currentText()
        order_type=self.comboBox_2.currentText()
        code=self.lineEdit.text()
        hoga=self.comboBox_3.currentText()
        num=self.spinBox.value()
        price=self.spinBox_2.value()

        self.kiwoom.send_order("send_order_req","0101",account, order_type_lookup[order_type],code, num, price, hoga_lookup[hoga],"")

    def check_balance(self):
        self.kiwoom.reset_opw00018_output()
        account_number=self.kiwoom.get_login_info("ACCNO")
        account_number=account_number.split(';')[0]
        
        self.kiwoom.set_input_value("계좌번호", account_number)
        self.kiwoom.comm_rq_data("opw00018_req","opw00018", 0, "2000")
        
        while self.kiwoom.remained_data:
            time.sleep(0.2)
            self.kiwoom.set_input_value("계좌번호", account_number)
            self.kiwoom.comm_rq_data("opw00018_req","opw00018", 0, "2000")
            
        #opw0001
        self.kiwoom.set_input_value("계좌번호", account_number)
        self.kiwoom.comm_rq_data("opw00001_req","opw00001", 0, "2000")
        
        #balance
        #예수금
        item=QTableWidgetItem(self.kiwoom.d2_deposit)
        item.setTextAlignment(Qt.AlignVCenter|Qt.AlignRight)
        self.tableWidget.setItem(0,0,item)
        
        #총매입, 총평가, 총손익, 총수익율, 추정자산
        for i in range(1,6):
                item=QTableWidgetItem(self.kiwoom.opw00018_output['single'][i-1])
                item.setTextAlignment(Qt.AlignVCenter|Qt.AlignRight)
                self.tableWidget.setItem(0,i,item)
        self.tableWidget.resizeRowsToContents()
        
        #종목 리스트
        item_count=len(self.kiwoom.opw00018_output['multi'])
        self.tableWidget_2.setRowCount(item_count)
        
        for j in range(item_count):
            row=self.kiwoom.opw00018_output['multi'][j]
            for i in range(len(row)):
                item=QTableWidgetItem(row[i])
                item.setTextAlignment(Qt.AlignVCenter|Qt.AlignRight)
                self.tableWidget_2.setItem(j,i,item)
        
        self.tableWidget_2.resizeRowsToContents()
    
    def timeout2(self):
        if self.checkBox.isChecked():
            self.check_balance()
        
        
    def query_deposit(self):
        self.kiwoom.set_input_value("계좌번호", "8037555511")
        self.kiwoom.set_input_value("비밀번호","1725")
        self.kiwoom.comm_rq_data("opw00001_req", "opw00001",0,"3")

        print(self.kiwoom.d2_deposit)
        
    def get_amount(self):
        account_number=self.kiwoom.get_login_info("ACCNO")
        account_number=account_number.split(';')[0]
        self.kiwoom.set_input_value("계좌번호", account_number)
        self.kiwoom.comm_rq_data("opw00018_req","opw00018", 0, "2000")


importing Jupyter notebook from Kiwoom.ipynb


In [2]:
if __name__=="__main__":
    app=QApplication(sys.argv)
    myWindow=MyWindow()
    myWindow.show()
    app.exec_()

connected
0
0149144
카카오                                  
10
0
0
0149145
키움증권                                
10
0
0
0149146
카카오                                  
10
0
0
0149147
키움증권                                
10
0
1

카카오                                  


1

키움증권                                


0
0149145
키움증권                                
10
0
1

키움증권                                


0
0149145
키움증권                                
10
0
1

키움증권                                


0
0149145
키움증권                                
10
0
1

키움증권                                


0
0149144
카카오                                  
10
0
0
0149146
카카오                                  
10
0
1

카카오                                  


1

카카오                                  


0
0149145
키움증권                                
10
0
1

키움증권                                


0
0149145
키움증권                                
10
0
1

키움증권                                


0
0149147
키움증권                   